## Understanding OmniAuth Keycloak Middleware

This example is part of a deeper study on [OmniAuth Keycloak Middleware](https://github.com/ccrockett/omniauth-keycloak), which enables Ruby applications to authenticate via Keycloak (an open-source Identity and Access Management solution).
Key Resources for a Complete Understanding

    omniauth-keycloak

        The official GitHub repository for the OmniAuth Keycloak strategy.

        Includes setup instructions, configuration options, and usage examples.

    Keycloak + OmniAuth Integration Guide (Coming Soon)

        A step-by-step tutorial covering:

            Keycloak realm & client setup

            OmniAuth middleware configuration

            User authentication flow in Ruby apps

        (Check back soon for the full guide!)

#### Why This Matters

    Secure Authentication: Keycloak provides OAuth 2.0 / OpenID Connect (OIDC) support.

    Ruby Integration: OmniAuth simplifies adding Keycloak-based login to Rails/Sinatra apps.

    Customizable: Supports role-based access control (RBAC), token validation, and user profile management.

If you're working with Keycloak + Ruby, reviewing these resources will help clarify the authentication flow and middleware behavior.


In [27]:
require 'net/http'
require 'json'
require 'uri'

url = URI("http://localhost:8080/realms/quickstart/.well-known/openid-configuration")

response = Net::HTTP.get_response(url)

if response.is_a?(Net::HTTPSuccess)
  data = JSON.parse(response.body)
  puts JSON.pretty_generate(data)
else
  puts "Request failed: #{response.code} #{response.message}"
end


{
  "issuer": "http://localhost:8080/realms/quickstart",
  "authorization_endpoint": "http://localhost:8080/realms/quickstart/protocol/openid-connect/auth",
  "token_endpoint": "http://localhost:8080/realms/quickstart/protocol/openid-connect/token",
  "introspection_endpoint": "http://localhost:8080/realms/quickstart/protocol/openid-connect/token/introspect",
  "userinfo_endpoint": "http://localhost:8080/realms/quickstart/protocol/openid-connect/userinfo",
  "end_session_endpoint": "http://localhost:8080/realms/quickstart/protocol/openid-connect/logout",
  "frontchannel_logout_session_supported": true,
  "frontchannel_logout_supported": true,
  "jwks_uri": "http://localhost:8080/realms/quickstart/protocol/openid-connect/certs",
  "check_session_iframe": "http://localhost:8080/realms/quickstart/protocol/openid-connect/login-status-iframe.html",
  "grant_types_supported": [
    "authorization_code",
    "client_credentials",
    "implicit",
    "password",
    "refresh_token",
    "urn:i

In [10]:
url = URI("http://localhost:8080/realms/quickstart/protocol/openid-connect/certs")
response = Net::HTTP.get_response(url)

if response.is_a?(Net::HTTPSuccess)
  data = JSON.parse(response.body)
  puts JSON.pretty_generate(data)
else
  puts "Request failed: #{response.code} #{response.message}"
end

{
  "keys": [
    {
      "kid": "mjSZU9heZBNKHkfZZBRIOlaf4ls6st6ltkQ1kF7BRe8",
      "kty": "RSA",
      "alg": "RSA-OAEP",
      "use": "enc",
      "x5c": [
        "MIICozCCAYsCBgGYgCvYHDANBgkqhkiG9w0BAQsFADAVMRMwEQYDVQQDDApxdWlja3N0YXJ0MB4XDTI1MDgwNjE2MTUxM1oXDTM1MDgwNjE2MTY1M1owFTETMBEGA1UEAwwKcXVpY2tzdGFydDCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBALWxpmPoJnAuaxa6+3DU/JqWkevL+CbucohN9F6zmU4FBs8BiBuzJKK4/FYJTVj0c5a9yUhpoJTCZEBrOEvLRRoqgnVbTFRA+JG5d3HPQZWhRb1d4+KKIxeNKEqMeckpf0luD4q9X6xELWGT1SwFkD13iOtSF7spBjLV6nnSx0LaCddRHEPaIy2b+rXVhWgdj0Gtl7vUQ+XzzaGolNx94a1MWz5y67CdhpI9OADH1gCWt2JHsOJLrCIr6LBx1KMKRSFq3gLZhgZxYAaspjjO97PJlBSLPDHOiIZe2xfCmBk3RG73gmLZU3VfrG1vIf2/PesfArY8D+9yQhT8ELCzrOMCAwEAATANBgkqhkiG9w0BAQsFAAOCAQEAR41tNR9mTyNL/c+qWPYA0F9q9/eC9dD8WuXp9gu+Qheq12FvR4kb4UlZN3RdiyfkVZ5jE1kYpR2SLixj51Rnisq96MopzOhbZyuOanC1qkcBJBsW+EppgbbW/JeSKzkzUNbbRcsp6bOKsbAexwY9UxSB2AE9Uy/g3nNssKkbqhRz00IBnoG8EKLmRDSZsuTNQH+fyLEd+IYhvTsxs3wUddGuEajYY9JOkzlpb4Asa2Sah+rZS7Al1HfSZEyR3o81DPCj2tfYJxL

#### First Go to keycloak dashboard and enable:
```
 Capability config
    Service accounts roles
```
Make sure you have `alice` user with password `password`

In [26]:
require 'net/http'
require 'uri'
require 'json'

# Step 1: Get access token via client credentials
token_uri = URI("http://localhost:8080/realms/quickstart/protocol/openid-connect/token")
token_res = Net::HTTP.post_form(token_uri, {
  "grant_type" => "password", # "client_credentials" -> for client itself, not for a user 
  "client_id" => "test-cli",
  "client_secret" => "38EdoDYcJmys5NENQ6WeNk7AIOwOlrLx",
  "username" => "alice",
  "password" => "password",
   "scope" => "openid profile email"  # ← Add this
})

token_data = JSON.parse(token_res.body)
access_token = token_data["access_token"]

if access_token.nil? || access_token.empty?
  puts "Failed to get access token: #{token_res.body}"
else
  puts "Access token: #{access_token}"

  # Step 2: Call UserInfo endpoint with Authorization header
  userinfo_uri = URI("http://localhost:8080/realms/quickstart/protocol/openid-connect/userinfo")
  
  # Create HTTP client
  http = Net::HTTP.new(userinfo_uri.host, userinfo_uri.port)
  
  request = Net::HTTP::Get.new(userinfo_uri)
  request["Authorization"] = "Bearer #{access_token}"
  
  response = http.request(request)

  if response.is_a?(Net::HTTPSuccess)
    data = JSON.parse(response.body)
    puts "UserInfo response:"
    puts JSON.pretty_generate(data)
  else
    puts "Request failed: #{response.code} #{response.message}"
    puts "Body: #{response.body}"
  end
end

Access token: eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICI1NWZyMnh4cWtidEF3RlZpZXZQWEdyVXdmTW16R2dMMmhXX3c0bEhwSmk0In0.eyJleHAiOjE3NTUxODk5NzksImlhdCI6MTc1NTE4OTY3OSwianRpIjoib25ydHJvOmQxZWQ2NTkxLWUyNzYtMGFjNC1lODJmLWM4Y2IyMzU3NzRjNCIsImlzcyI6Imh0dHA6Ly9sb2NhbGhvc3Q6ODA4MC9yZWFsbXMvcXVpY2tzdGFydCIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI4ZWIzZjQwMS0yNWIxLTQ1MzctYWUyYy05MjQ1ZDc5OWJjNjkiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJ0ZXN0LWNsaSIsInNpZCI6IjM3YmExZWIxLTI2NGYtNDVjMC1iODhkLTA2YjlhMTQ5MTllYyIsImFjciI6IjEiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cDovL2xvY2FsaG9zdDozMDAwIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsImRlZmF1bHQtcm9sZXMtcXVpY2tzdGFydCIsInVtYV9hdXRob3JpemF0aW9uIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgcHJvZmlsZSBlbWFpbCIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiQWxpY2UgTGlkZGVsIiwicHJlZmVycmVkX3VzZXJuYW1lIjoiYWxpY2UiLCJnaXZlbl9uYW1lIjoiQWxpY2UiLCJmYW1pbHlfbmFtZSI6Ik

### Note: 

#### Important detail with the UserInfo endpoint:

The `client_credentials` grant typically returns a token for the client itself, not for a user.

The UserInfo endpoint requires a token that represents a user, usually obtained via `authorization code` flow.

In [28]:
puts "That is it! Thanks!"

That is it! Thanks!
